# GloVe: Global Vectors for Word Representation (25 points)

[GloVe](http://nlp.stanford.edu/projects/glove/) \[[PDF](http://nlp.stanford.edu/pubs/glove.pdf)\] is (yet) another way to train word vectors.  Its main advantage relative to Word2Vec is its training speed.

## Approach
The intuition of the GloVe approach to training word vectors is as follows:

1. From the training data, estimate $P(k | word)$.
2. Notice that some words, $k$, are far more common than others in the context of $word$.
3. In particular, in the table below, notice in the bottom row that $k$'s that are related to ice (vs. steam) result in quite large numbers where as those related to steam (vs. ice) are incredibly low.  Unrelated numbers are about 1.0.

<img src="glove_table.png">

At a high level then training $F(w_i, w_j, \tilde{w}_k) = \frac{P_{ij}}{P_{jk}}$ seems like a useful thing to do.  In this case, F is a (simple) neural network accepting word vectors $w_i$ and $w_j$ for words $i$ and $j$, and a context vector $\tilde{w}_k$ for word $k$.

With some reasonable assumptions about desireable properties of vector embeddings (see Section 3 of the paper), the authors make this more concrete and simplify to a simple objective function based directly on the cooccurrence matrix instead of probabilities:

$$J = \sum\limits_{i,j}^V f(X_{ij})(w_i^T\tilde{w}_j + b_i + \tilde{b}_j - log(X_{ij}))^2$$

where $f(.)$ is the weight of the $j$'th word appearing in the $i$th word's context window $X_{ij}$ times.  This weighting function is described in detail immediately before Equation (9) in the paper.

Note that $f(0) = 0$ pairs $i,j$ where $X_{ij} = 0$ can be skipped in the sum above.

## vs. Word2Vec
Similar to Word2Vec, GloVe embeds words in a vector space based on the "[company it keeps](https://en.wikipedia.org/wiki/John_Rupert_Firth)" - based on cooccurrance between words in small context windows.  Unlike Word2Vec which repeatedly iterates over the training data one context window at a time, GloVe does a single pass over the training data to collect cooccurrance statistics.  GloVe then trains entirely based on this table of counts.

## The Plan
In this assignment, you are going to train GloVe models and visualize them.

1. Parsing utilities.

2. Phrases.  Implement Section 4 (Equation 6) of the [Word2Vec](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) paper.  This allows us to treat "los angeles" as a single item in our vocabulary.

3. Implement TensorFlow for GloVe & Train embeddings.

5. Visualize embeddings.


As usual, we begin by importing some useful libraries.

In [2]:
import glove
import glove_test
import nltk
import numpy as np
import random
import tensorflow as tf
import time
import unittest
import word_stream
import word_stream_test
import word_utils
reload(glove)
reload(glove_test)
reload(word_stream)
reload(word_stream_test)
reload(word_utils)

<module 'word_utils' from 'word_utils.pyc'>

In [3]:
# Lower casing sometimes causes more harm than good,
# but we do it here anyways in absence of more careful normalization.

# The Brown corpus is only 1m tokens.  Test your code with this, then if you want, run with Wikipedia.

# words = [w.lower() for w in nltk.corpus.brown.words()]

# Use first billion bytes of Wikipedia, consisting of 17m tokens.  While this produces better
# embeddings, all of the code runs correspondingly longer.  We recommend getting everything to work
# with the Brown corpus before trying this.

# If you are going to try this using Google Compute Engine, we recommend using the
# n1-highcpu-16 (16 vCPUs, 14.4 GB memory) version.

# You should not spend time debugging on this instance though or you will find yourself
# without GCE credit!

words = open('text8').read().split()

The data looks like this:

In [4]:
words[:10], len(words)

(['anarchism',
  'originated',
  'as',
  'a',
  'term',
  'of',
  'abuse',
  'first',
  'used',
  'against'],
 17005207)

## 1.  Parsing Utilities (8 points)

In **word_stream.py** (find this in the same directory as this notebook), implement:
1.  context_windows
2.  cooccurrence_table

You may find it helpful to run the follow unit tests to check your code.

In [5]:
reload(word_stream)
reload(word_stream_test)
unittest.TextTestRunner(verbosity=2).run(
    unittest.TestLoader().loadTestsFromName(
        'TestWordStreams.test_context_windows', word_stream_test))

test_context_windows (word_stream_test.TestWordStreams) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.202s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [6]:
reload(word_stream)
reload(word_stream_test)
unittest.TextTestRunner(verbosity=2).run(
    unittest.TestLoader().loadTestsFromName(
        'TestWordStreams.test_cooccurrence_table', word_stream_test))

test_cooccurrence_table (word_stream_test.TestWordStreams) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## 2.  Phrases (4 points)

Implement the function **score_bigram** in **word_stream.py**.
Specifically, read the introduction to Section 4 of the [Word2Vec](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) paper and implement Equation 6.

In [7]:
reload(word_stream)
reload(word_stream_test)
unittest.TextTestRunner(verbosity=2).run(
    unittest.TestLoader().loadTestsFromName(
        'TestWordStreams.test_score_bigram', word_stream_test))

test_score_bigram (word_stream_test.TestWordStreams) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

Use your scoring code to see the best scoring phrases.

In [8]:
delta = 100  # You can play with this hyperparameter to see how it affects the results.
unigrams, bigrams = word_utils.unigram_and_bigram_counts(words)
scored_bigrams = sorted(
    [(word_stream.score_bigram(bigram, unigrams, bigrams, delta), bigram) for bigram in bigrams],
    reverse=True)
scored_bigrams[0:500]

[(0.001957730812013348, ('burkina', 'faso')),
 (0.0018769551616266945, ('buenos', 'aires')),
 (0.0015462384775498067, ('costa', 'rica')),
 (0.001476912716754159, ('fran', 'ois')),
 (0.0014698871762275545, ('sri', 'lanka')),
 (0.0013311013855555331, ('las', 'vegas')),
 (0.00129174343010821, ('coca', 'cola')),
 (0.0012493492972410203, ('nova', 'scotia')),
 (0.0012483640390617136, ('saudi', 'arabia')),
 (0.0011100906440921615, ('monty', 'python')),
 (0.0010555160864711254, ('puerto', 'rico')),
 (0.001049632628579997, ('guant', 'namo')),
 (0.0010233918128654971, ('gda', 'sk')),
 (0.0008804754567466431, ('mario', 'kart')),
 (0.0008553519070943142, ('hip', 'hop')),
 (0.0007207648654192401, ('import', 'insubstantial')),
 (0.0007022071041776631, ('los', 'angeles')),
 (0.0006783348664121336, ('hong', 'kong')),
 (0.000629765412383887, ('walt', 'disney')),
 (0.0006038647342995169, ('ann', 'arbor')),
 (0.0005259159703149653, ('bene', 'gesserit')),
 (0.0005191225884396119, ('ronald', 'reagan')),
 (

This next cell uses the scores computed above and calls grouped_stream which takes a list of words and a set of n-grams and returns the list of words with those n-grams combined into single tokens.

e.g. ['the', 'supreme', 'court'] => ['the', 'supreme_court']

(You can find more examples in the tests for the function in word_stream_test.py.)

In [9]:
# You can leave this cell alone.

# If you want to come back afterwards, you can experiment with different phrase_thresholds.
# You should set phrase_threshold to a value at which the bigrams in the previous
# output start looking less tightly coupled.  grouped_stream below will re-tokenize
# a stream of words to consider bigrams scoring above phrase_threshold as a single token.

phrase_threshold = 1.0
phrases = [bigram for score, bigram in scored_bigrams if score >= phrase_threshold]
words = word_utils.grouped_stream(words, phrases)

In [19]:
# Cleanup some variables to recover some memory.
del unigrams
del bigrams
del scored_bigrams
del phrases

## 3. TensorFlow for GloVe

### Cooccurrence Table
In this section, we first build the cooccurrence table with context window of size C.

In [10]:
VOCAB_SIZE = 20000  # Amount of vocabulary to keep.  Lower frequency words are mapped to <UNK> (word id: 0).

# Map each of the words to a wordid.  Only the most popular VOCAB_SIZE words are kept.
vocabulary = word_utils.Vocabulary(words, VOCAB_SIZE)

In [11]:
# Convert the word stream to wordids.
# We need to do this because the TensorFlow code you will write in the
# next section will use an API that expects indexes into the embedding
# matrix, not words.
wordids = [vocabulary.to_id(word) for word in words]


In [12]:
# Building the cooccurrence table takes a considerable amount of time
# with the Wikipedia set.

C = 10  # Context window size.
ctable = word_stream.cooccurrence_table(wordids, C)


In [13]:
# Output top words occurring within the same context.
# If everything has worked properly, you should see a considerable number of "<UNK>", "the", "of", and numbers.
sorted([(vocabulary.to_word(word), vocabulary.to_word(context_word), count) for word, context_word, count in ctable if count > len(words) / 100], key=lambda x: x[2], reverse=True)[:10]

[('<UNK>', '<UNK>', 706138.1285682213),
 ('the', 'of', 374048.6742070548),
 ('of', 'the', 374048.57420705474),
 ('<UNK>', 'the', 342798.24642992363),
 ('the', '<UNK>', 342798.24642992363),
 ('the', 'the', 269496.53413024516),
 ('zero', 'zero', 264773.4888895979),
 ('one', 'nine', 257475.82539757458),
 ('nine', 'one', 257475.71428646345),
 ('<UNK>', 'and', 193557.81507994025)]

In [14]:
# Shard the table into word lists, context word lists and corresponding counts.
# We are going to provide these to TensorFlow as their own entry in the feed_dict,
# so we do this separation once, up front.
ctable_wids = np.array([word for word, _, _ in ctable])
ctable_cwids = np.array([context_word for _, context_word, _ in ctable])
ctable_counts = np.array([count for _, _, count in ctable])

In [15]:
# Here is what our final data looks like.  It's similar to the table two
# cells previous, except instead of words, there are wordids.
zip(ctable_wids[:5], ctable_cwids[:5], ctable_counts[:5])

[(0, 0, 706138.12856822135),
 (0, 1, 107923.21904752152),
 (0, 2, 117.23730158730162),
 (0, 3, 29.944841269841259),
 (0, 4, 50.867857142857197)]

### TensorFlow Graph Setup (12 points)

Complete the functions in **glove.py** using the TensorFlow API and then run the corresponding tests in the cells below.

In [16]:
reload(glove)
reload(glove_test)
unittest.TextTestRunner(verbosity=2).run(
    unittest.TestLoader().loadTestsFromName(
        'TestGlove.test_embedding_lookup', glove_test))

test_embedding_lookup (glove_test.TestGlove) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.170s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [17]:
reload(glove)
reload(glove_test)
unittest.TextTestRunner(verbosity=2).run(
    unittest.TestLoader().loadTestsFromName(
        'TestGlove.test_example_weight', glove_test))

test_example_weight (glove_test.TestGlove) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.044s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [18]:
reload(glove)
reload(glove_test)
unittest.TextTestRunner(verbosity=2).run(
    unittest.TestLoader().loadTestsFromName(
        'TestGlove.test_loss', glove_test))

test_loss (glove_test.TestGlove) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.095s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [22]:
# Hyperparameters.

# You may want to shrink num_examples_to_train to finish debugging
# and only run it this long once you are training on Wikipedia.
learning_rate = 0.003
num_examples_to_train = 300000000
#num_examples_to_train = 3000
batch_size = 100
embedding_dim = 300

# Construct the training graph.
tf.reset_default_graph()

wids_ph = tf.placeholder(tf.int32, shape=[None])
c_wids_ph = tf.placeholder(tf.int32, shape=[None])
counts_ph = tf.placeholder(tf.float32, shape=[None])

with tf.variable_scope('word_embeddings'):
    word_embeddings, word_bias, word_embed_matrix = (
        glove.wordids_to_tensors(wids_ph, embedding_dim, vocabulary.size()))
with tf.variable_scope('word_context_embeddings'):
    word_context_embeddings, word_context_bias, word_context_embed_matrix = (
        glove.wordids_to_tensors(c_wids_ph, embedding_dim, vocabulary.size()))
    
losses = glove.loss(
    word_embeddings, word_bias, word_context_embeddings, word_context_bias,
    tf.cast(counts_ph, tf.float32))
loss = tf.reduce_mean(losses)

# Adam is similar to AdaGrad in that it handles sparse gradients well.
# Specifically, you may imagine that some words appear with more context
# words than others and with bigger counts.  They therefore are updated
# more often and more aggressively (remember the weighting function
# you implemented).  Adam backs off updating parameters that it has already
# significantly moved around.  (intuitively: the 500th time you backprop
# into "the", you probably don't have a lot more information to add).
#
# Here is the original University of Toronto paper detailing the word
# done in collaboration with Google DeepMind.
# https://arxiv.org/pdf/1412.6980v8.pdf
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

### Train the Embeddings

In [23]:
# Train the embeddings.
# Set up the session & initialize variables.
sess = tf.Session()
sess.run(tf.initialize_all_variables())

In [ ]:
# Important note:  You do not need to run this cell to completion.
# Let it train for 30 minutes or so, then interrupt the kernel and see how good
# the nearest-neighbors results look.  Run this cell again to pick up from where
# you left off.

# An hour on the recommended GCE cloud instance gets reasonably good results.
# Two hours cleans up the vectors beautifully.

REPORT_LOSS_EVERY = 1000
EVAL_BATCH_SIZE = 5000

indexes = range(len(ctable_wids))

def make_feed_dict(feed_dict_batch_size):
    batch_idx = random.sample(indexes, feed_dict_batch_size)
    batch_wids = ctable_wids[batch_idx]
    batch_cwids = ctable_cwids[batch_idx]
    batch_counts = ctable_counts[batch_idx]
    return {
        wids_ph: batch_wids,
        c_wids_ph: batch_cwids,
        counts_ph: batch_counts
    }

num_batches = num_examples_to_train / batch_size + 1

print '# training examples:', len(ctable_wids)
print '# of epochs:', 1.0 * num_examples_to_train / len(ctable_wids)
print '# batches:', num_batches
print 'Initial loss:', sess.run(loss, feed_dict=make_feed_dict(EVAL_BATCH_SIZE))

current_timer = None
for batch in xrange(num_batches):
    # Train based on randomly sampled batches of examples.
    loss_val, _ = sess.run([loss, train_op], feed_dict=make_feed_dict(batch_size))
    
    # Do some basic reporting as training progresses.
    if batch % REPORT_LOSS_EVERY == 0:
        if current_timer:
            remaining_reporting_cycles = 1.0 * (num_batches - batch) / REPORT_LOSS_EVERY
            cycle_time = time.time() - current_timer
            print 'Expected time left:', remaining_reporting_cycles * cycle_time / 60 / 60, 'hours (', cycle_time, 'seconds per', REPORT_LOSS_EVERY, 'batches).'
        current_timer = time.time()
            
        print batch, ':', sess.run(loss, feed_dict=make_feed_dict(EVAL_BATCH_SIZE))

# training examples: 31302717
# of epochs: 9.58383261108
# batches: 3000001
Initial loss: 1.61115
0 : 1.87783
Expected time left: 48.6420211555 hours ( 58.3898692131 seconds per 1000 batches).
1000 : 1.63667
Expected time left: 54.3404595704 hours ( 65.2520310879 seconds per 1000 batches).
2000 : 2.07162
Expected time left: 55.5742327687 hours ( 66.7558128834 seconds per 1000 batches).
3000 : 1.08939
Expected time left: 56.2798013419 hours ( 67.6259069443 seconds per 1000 batches).
4000 : 1.19348
Expected time left: 55.9276270815 hours ( 67.2251720428 seconds per 1000 batches).
5000 : 1.45329
Expected time left: 56.7409197772 hours ( 68.2255320549 seconds per 1000 batches).
6000 : 1.58094
Expected time left: 56.1176747814 hours ( 67.4986841679 seconds per 1000 batches).
7000 : 0.880944
Expected time left: 56.2169902893 hours ( 67.6407411098 seconds per 1000 batches).
8000 : 2.48954
Expected time left: 56.9559065423 hours ( 68.5527231693 seconds per 1000 batches).
9000 : 2.23745
Expecte

## Play! (1 point)

Congratulations!  You now have some embeddings.  We only trained a short while over not a lot of text, but these still work reasonably well.

If you want more compelling vectors, scroll back up to the top of this notebook and follow the instructions to switch the data source to Wikipedia and execute it again.  Note:  training these vectors for a long time is **not required**, but since you've gotten this far, it takes almost no additional effort to see the result of your hard work below.  The longer you run on the Wikipedia set, the nicer your word vectors will be.  As noted in the previous cell, you can let it run for a while, interrupt the kernel, see how things look, and then run that cell again to have it pick up from where it left off.

We have a number of suggestions below to get you started exploring the space.  Feel free to try some of your own.

In [48]:
def find_nn_cos(v, Wv, k=10):
    """Find nearest neighbors, by cosine distance."""
    Z = np.linalg.norm(Wv, axis=1) * np.linalg.norm(v)
    ds = np.dot(Wv, v.T) / Z
    nns = np.argsort(-1*ds)[:k]  # sort descending, take best
    return nns, ds[nns]  # word indices, distances

def show_nns(v, Wv, vocabulary, k=10):
    print "Nearest neighbors:"
    for i, d in zip(*find_nn_cos(v, Wv, k)):
        w = vocabulary.to_word(i)
        print "%.03f : \"%s\"" % (d, w)
        
def word_show_nns(word, Wv, vocabulary, k=10):
    show_nns(Wv[vocabulary.to_id(word)], Wv, vocabulary, k)

In [49]:
word_embed_matrix_val, word_context_embed_matrix_val = sess.run([word_embed_matrix, word_context_embed_matrix])

# As per the paper, we take the average of the word's vector when it's the center word of the window
# and the vector when it's found in the context.
#
# There is some (handwave-y) motivation for why we do this in section 4.2 of GloVe.
Wv = word_embed_matrix_val + word_context_embed_matrix_val

In [50]:
word_show_nns('one', Wv, vocabulary)

Nearest neighbors:
1.000 : "one"
0.216 : "frequencies"
0.205 : "perceive"
0.203 : "dictate"
0.196 : "governmental"
0.194 : "lacks"
0.192 : "northfield"
0.189 : "enthusiastically"
0.187 : "around"
0.186 : "keith's"


In [ ]:
word_show_nns('king', Wv, vocabulary)

In [ ]:
word_show_nns('car', Wv, vocabulary)

In [ ]:
word_show_nns('computer', Wv, vocabulary)

In [ ]:
word_show_nns('college', Wv, vocabulary)